In [13]:
#| default_exp tables_to_viewer

In [14]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
#| export

from pathlib import Path
from copy import deepcopy

import pandas as pd
from pydantic_settings import CliApp

from clip_plot.configuration import Cfg
from clip_plot.utils import timestamp
from clip_plot.web_config import copy_web_assets
from clip_plot.images import ImageFactory
from clip_plot.metadata import get_manifest, write_metadata
from clip_plot.images import create_atlases_and_thumbs, ImageFactory


from clip_plot.from_tables import table_to_meta, cat_tables

In [16]:
#| export

_D = Cfg() # defaults

In [17]:
#| export

def tables_to_viewer_pipeline(
    tables: list[Path] = deepcopy(_D.paths.tables),
    output_dir: Path = deepcopy(_D.paths.output_dir),
    tagline: str = deepcopy(_D.tagline),
    logo: str = deepcopy(_D.logo),
):
    """
    From a set of tables with embed_path and image_path columns,
    generate a clipplot viewer
    """
    print(timestamp(), "Starting image processing pipeline.")
    df = cat_tables(tables, table_format=tables[0].suffix)
    metadata = table_to_meta(df)

    copy_web_assets(out_dir=output_dir.as_posix(),
                    tagline=tagline, logo=logo)
    image_engine = ImageFactory(data_dir=output_dir/"data",
                                meta_dir=None,
                                img_path=df["image_path"].tolist(),
                                )

    write_metadata(image_engine)
    # embeddings = //

    kwargs["atlas_dir"], atlas_data = create_atlases_and_thumbs(imageEngine, kwargs["plot_id"], kwargs["use_cache"])
    get_manifest(imageEngine, atlas_data, **kwargs)
    print(timestamp(), "Done!")




In [18]:
#| export

def tables_to_viewer_cli():
    """
    run embed_to_tables with CLI defined
    from pydantic-settings configuration object
    """
    cfg = CliApp.run(Cfg)
    tables_to_viewer_pipeline(
        tables=cfg.paths.tables,
        tagline=cfg.viewer.tagline,
        logo=cfg.viewer.logo,
        output_dir=cfg.paths.output_dir,
        table_format=cfg.paths.table_format
    )

In [19]:
#|hide
import nbdev; nbdev.nbdev_export()